In [4]:
from manim import *

config.media_width = "75%"
config.verbosity = "WARNING"

In [5]:
from scipy.spatial import distance
import numpy as np

# Distance Metrics Animation

## Intro

In [3]:
what_is_distance_text = """Distance is the length between two things. 
Usually, we think of distance as a line connecting two points.
Instead of A and B, lets add some context and call them Home and School.
The distance between Home and School is 2 kilometers. That's the length of the straight line between them.
Easiest, the most straight-forward of the distances.
However it is not always the best choice. 
Flying from Home to School is quite unlikely (unfortunately), so we may want to measure the length of the streets.
Speaking of flying, going to vacation on the other side of the world involves a flight in not so straight line ether.
But back to school. The distance to School is 2 kilometers, also 3 kilometers, and also ten bus stops, or 40 minutes.
These are all valid measurements, and which one we choose depends on the task at hand."""

%%manim -qm Intro


class Intro(Scene):

    def construct(self):
        
        # Text utils
        def text_generator(text):
            for new_line in text.split("\n"):
                yield new_line       

        def next_text_line(text_gen, text_obj = None, wait_time = 0):
            if text_obj:
                self.play(Unwrite(text_obj, reverse = False))

            text_obj = Tex(next(text_gen), )\
                .scale(0.9)\
                .shift(DOWN * 3)

            self.play(Write(text_obj))
            self.wait(wait_time)
            
            return text_obj


        # Title
        title = Text("What is distance?")\
            .scale(1.2)\
            .to_edge(UL)
        self.play(Write(title))
 
        # Narrative generator         
        narrative = text_generator(what_is_distance_text)
        

        # Points A and B
        point_home = Dot(point = [-1, -1, 0])
        point_a = Text("A")\
            .next_to(point_home)\
            .shift(LEFT)    
        point_school = Dot(point = [3, 2, 0])
        point_b = Text("B")\
            .next_to(point_school)
        
        text = next_text_line(narrative)
        self.play(Create(point_home), Create(point_a))
        self.play(Create(point_school), Create(point_b))      
        



        # Straight line A to B
        line = Line(start=point_home, end=point_school)

        text = next_text_line(narrative, text_obj = text)
        self.play(Create(line))

           
        
        # Home and School
        home = ImageMobject("icons/house.jpg")
        home.height = 0.6
        home.next_to(point_home, LEFT)

        school = ImageMobject("icons/school.jpg")
        school.height = 0.6
        school.next_to(point_school)


        text = next_text_line(narrative, text_obj = text)
        self.play(Uncreate(point_a), Uncreate(point_b))   
        self.add(home, school)




        # self.play(Unwrite(text, reverse = False))
        # text = next_text_line(narrative)

        # self.play(Write(text))

        # self.wait(3)
        # # next_text_line()
        # # next_text_line()

        self.wait(3)
        
